In [1]:
import json

In [2]:
with open('recipes.json', encoding="utf8") as f:
    data = json.load(f)


In [3]:
from pandas.io.json import json_normalize
df = json_normalize(data[2]['data'])

In [4]:
df.head()

,id,name,ingredients,method,image,video
0,1,KALAKAND SWEET / MITHAI,Milk powder 2 cups~Sugar 1/2 cup or to taste~...,"In a heat resistant bowl add powder milk, crea...",http://www.ainycooks.com/wp-content/uploads/20...,https://www.youtube.com/embed/8ApA0f7vs00
1,2,ARIZONA CHICKEN STEAK,Chicken fillets 2 flattened~Salt to tatse~Whit...,"Marinate chicken fillet with salt, pepper, mus...",http://www.ainycooks.com/wp-content/uploads/20...,None
2,3,CHICKEN CREAMY PIE,Chicken boiled and shredded 1 cup~Onion chopp...,"In a pan add butter and onion, stir it for few...",http://www.ainycooks.com/wp-content/uploads/20...,None
3,4,2 INGREDIENTS CHOCOLATE MOUSSE,Cooking Chocolate / Milk chocolate 200g~Whipp...,Add 1/2 cup cream in chocolate and microwave i...,http://www.ainycooks.com/wp-content/uploads/20...,None
4,5,KARACHI FAMOUS STUDENT BIRYANI,Rice Basmati 1/2 kg~Chicken 1/2 kg~Red chili ...,Soak rice for 20 minutes.~Boil rice and add sa...,http://www.ainycooks.com/wp-content/uploads/20...,None


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [7]:
import re

In [8]:
datas = []
for index, row in df.iterrows():
    ings = row['ingredients'].replace(' ','')
    ings = ings.split('~')
    data = {}
    data['recipie'] = row['name']
    data['ingredients'] = ings
    datas.append(data)

In [9]:
df = json_normalize(datas)

In [10]:
df.head()

,recipie,ingredients
0,KALAKAND SWEET / MITHAI,"[Milkpowder2cups, Sugar1/2cuportotaste, Cream2..."
1,ARIZONA CHICKEN STEAK,"[Chickenfillets2flattened, Salttotatse, Whitep..."
2,CHICKEN CREAMY PIE,"[Chickenboiledandshredded1cup, Onionchopped1me..."
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[CookingChocolate/Milkchocolate200g, Whippingc..."
4,KARACHI FAMOUS STUDENT BIRYANI,"[RiceBasmati1/2kg, Chicken1/2kg, Redchilipowde..."


In [11]:
new = []
for s in df['ingredients']:
    s = ' '.join(s)
    new.append(s)
df['ing'] = new
l=[]
for s in df['ing']:
    
    #Remove punctuations
    s=re.sub(r'\(.*oz.\)|tte|kg|crushed|crumbles|ground|minced|tsp|tbsp|as|required|for|powder|chopped|sliced|pinch|cup|/|ml|[0-9]',
                             '', 
                             s)
    
    #Remove Digits
    s=re.sub(r"(\d)", "", s)
    
    #Remove content inside paranthesis
    s=re.sub(r'\([^)]*\)', '', s)
    
    #Remove Brand Name
    s=re.sub(u'\w*\u2122', '', s)
    
    #Convert to lowercase
    s=s.lower()
    
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    s= ' '.join(filtered_sentence)
    
    l.append(s)
df['ing_mod']=l

In [12]:
df.head()

,recipie,ingredients,ing,ing_mod
0,KALAKAND SWEET / MITHAI,"[Milkpowder2cups, Sugar1/2cuportotaste, Cream2...",Milkpowder2cups Sugar1/2cuportotaste Cream200m...,milks sugarortotte cream nuts
1,ARIZONA CHICKEN STEAK,"[Chickenfillets2flattened, Salttotatse, Whitep...",Chickenfillets2flattened Salttotatse Whitepepp...,chickenfilletsflaned salttotatse whitepepper b...
2,CHICKEN CREAMY PIE,"[Chickenboiledandshredded1cup, Onionchopped1me...",Chickenboiledandshredded1cup Onionchopped1medi...,chickenboiledandshredded onionmedium bur black...
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[CookingChocolate/Milkchocolate200g, Whippingc...",CookingChocolate/Milkchocolate200g Whippingcre...,cookingchocolatemilkchocolateg whippingcreams
4,KARACHI FAMOUS STUDENT BIRYANI,"[RiceBasmati1/2kg, Chicken1/2kg, Redchilipowde...",RiceBasmati1/2kg Chicken1/2kg Redchilipowder1t...,ricebmati chicken redchili blackpepper whitepe...


In [13]:
labels = list()
for line in df.recipie:
    labels.append(line)
texts = list()
for line in df.ing_mod:
    texts.append( line )
lengths = list()
for text in texts:
    lengths.append( len( text.split() ) )
maxlen = max( lengths )
labels = np.array( labels )
texts = np.array( texts )

In [14]:
maxlen

58

In [15]:
len(labels)

1721

In [16]:
import tensorflow as tf

In [17]:
from sklearn import preprocessing

In [18]:
le = preprocessing.LabelEncoder()

In [34]:
le.fit(labels)

LabelEncoder()

In [20]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( texts )
tokenized_messages = tokenizer.texts_to_sequences( texts )
padded_messages = tf.keras.preprocessing.sequence.pad_sequences( tokenized_messages , maxlen )
onehot_labels = tf.keras.utils.to_categorical( list(le.transform(labels)) , num_classes=1721 )

X = padded_messages
Y = onehot_labels

print( X.shape )
print( Y.shape )
print( 'MESSAGE MAXLEN = {}'.format( maxlen ) )

output_path = 'processed_data/'
np.save( '{}x.npy'.format( output_path )  , X )
np.save( '{}y.npy'.format( output_path )  , Y )

with open( 'processed_data/word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )

print( 'Data processed.')

(1721, 58)
(1721, 1721)
MESSAGE MAXLEN = 58
Data processed.


In [21]:
with open('processed_data/vocab.txt' , 'w') as file:
    for key in tokenizer.word_index.keys():
        file.write(key+' '+str(tokenizer.word_index[key])+'\n')

In [22]:
with open('processed_data/labels.txt' , 'w') as file:
    for label in labels:
        file.write(label+'\n')

In [23]:
from tensorflow.python.keras.callbacks import TensorBoard
import numpy as np
import time

X = np.load( 'processed_data/x.npy')
Y = np.load( 'processed_data/y.npy')

print( X.shape )
print( Y.shape )

# classifier = Classifier( number_of_classes=1721 , maxlen=102 )
# #classifier.load_model( 'models/model.h5' )

# parameters = {
#     'batch_size' : 100 ,
#     'epochs' : 100 ,
#     'callbacks' : [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
#     'val_data' : ( test_X , test_Y )
# }
# classifier.fit( X , Y , parameters )
# classifier.save_model( 'models/model.h5')

# loss , accuracy = classifier.evaluate( test_X , test_Y )
# print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ) )


input_dim = X.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1721, activation='sigmoid'))


(1721, 58)
(1721, 1721)


In [24]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

history = model.fit(X, Y,
                    epochs=100,
                    verbose=True,
                    validation_data=(X, Y),
                    batch_size=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                590       
_________________________________________________________________
dense_2 (Dense)              (None, 1721)              18931     
Total params: 19,521
Trainable params: 19,521
Non-trainable params: 0
_________________________________________________________________
Train on 1721 samples, validate on 1721 samples
Epoch 1/100
1721/1721 [==============================] - 1s 536us/step - loss: 1.7418 - accuracy: 0.9320 - val_loss: 0.0688 - val_accuracy: 0.9944
Epoch 2/100
1721/1721 [==============================] - 1s 468us/step - loss: 0.0611 - accuracy: 0.9962 - val_loss: 0.0485 - val_accuracy: 0.9976
Epoch 3/100
1721/1721 [==============================] - 1s 346us/step - loss: 0.0480 - accuracy: 0.9979 - val_loss: 0.0398 - val_accuracy: 0.9984
Epoch 4/100
1721/1721 [

Epoch 52/100
1721/1721 [==============================] - 1s 403us/step - loss: 0.0055 - accuracy: 0.9993 - val_loss: 0.0045 - val_accuracy: 0.9993
Epoch 53/100
1721/1721 [==============================] - 1s 400us/step - loss: 0.0054 - accuracy: 0.9993 - val_loss: 0.0044 - val_accuracy: 0.9994
Epoch 54/100
1721/1721 [==============================] - 1s 404us/step - loss: 0.0052 - accuracy: 0.9993 - val_loss: 0.0043 - val_accuracy: 0.9993
Epoch 55/100
1721/1721 [==============================] - 1s 406us/step - loss: 0.0054 - accuracy: 0.9993 - val_loss: 0.0049 - val_accuracy: 0.9992
Epoch 56/100
1721/1721 [==============================] - 1s 404us/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.0045 - val_accuracy: 0.9993
Epoch 57/100
1721/1721 [==============================] - 1s 407us/step - loss: 0.0051 - accuracy: 0.9993 - val_loss: 0.0040 - val_accuracy: 0.9994
Epoch 58/100
1721/1721 [==============================] - 1s 403us/step - loss: 0.0046 - accuracy: 0.9993 - val_

In [25]:
model.save('processed_data/model.h5')

In [55]:
keras_model_path = "processed_data/model.h5"
import tensorflow as tf
converter =  tf.compat.v1.lite.TFLiteConverter.from_keras_model_file( keras_model_path)
tfmodel = converter.convert()
open ("model.tflite" , "wb") .write(tfmodel)

79348

In [48]:
from keras.models import load_model
 
# load model
model = load_model('processed_data/model.h5')

In [46]:
count = 0
true_count = 0
for index, row in df.iterrows():
    arr = tf.keras.preprocessing.sequence.pad_sequences( tokenizer.texts_to_sequences([row['ing_mod']]) , maxlen )
    res = model.predict_classes(arr)
    res = le.inverse_transform(res)[0]
    if res == row['recipie']:
        true_count -=- 1
    count -=- 1

In [47]:
count

1721

In [48]:
true_count

671

In [67]:
arr = tf.keras.preprocessing.sequence.pad_sequences( tokenizer.texts_to_sequences(['milks sugarortotte cream nuts']) , maxlen )
res = model.predict_classes(arr)
res = le.inverse_transform(res)[0]

In [68]:
res

'Chicken Burger with Damigras Sauce Recipe in Urdu'

In [69]:
model.predict(arr)[0][667]

1.013279e-06

In [65]:
model.predict_classes(arr)

array([667], dtype=int64)

In [58]:
new_model= tf.keras.models.load_model(filepath=keras_model_path)

In [59]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model)

79332

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ds= df
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['ing_mod'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['recipie'][i]) for i in similar_indices]

    results[row['recipie']] = similar_items[1:]
    
print('done!')


done!


In [79]:

def item(id):
    return ds.loc[ds['recipie'] == id]['ing_mod'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

recommend(item_id='KALAKAND SWEET / MITHAI', num=5)


Recommending 5 products similar to milks sugarortotte cream nuts...
-------
Recommended: chikoomediumsize milkgls sugarortotte cream (score:0.23318662151369013)
Recommended: mango milks sugarortotte pinchofsaffron iceneeded almondpistachiogarnish (score:0.19480188206886545)
Recommended: chocolatechipsorcookingchocolatesemisweets condensedmilkcan margarineoranybur nuts (score:0.0896473587376921)
Recommended: semolinasuji , soakinwater sugar water gheeor . cardamom- nuts (score:0.06835975674534608)
Recommended: wontonsamosawraps burbrushing eggbrushing nuts date sugar water (score:0.06629981053412458)


In [22]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

def read_data(path):
	ted_data = pd.read_csv(path)
	return ted_data

def tf_idf(keys, dataframe, label, min_videos=10):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe.loc[:, label])
    with open('tf_idf','wb') as f:
        pickle.dump(tfidf_matrix, f)
    
    with open('tf_idf_vec','wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    query = tfidf_vectorizer.transform([keys])
    cs = cosine_similarity(query, tfidf_matrix)
    similarity_list = cs[0]
    result_list = []
    x = similarity_list
    index = np.argwhere(x!=0)
    data = {}
    for ind in index:
        data[ind[0]] = x[ind][0]
    return data

In [23]:
y = tf_idf('milk',df, 'ing_mod')

In [24]:
y = {k: v for k, v in sorted(y.items(), key=lambda item: item[1], reverse=True)}

In [25]:
for k in y.keys():
    print(df.iloc[k]['recipie'])

Home Made Khoya
DULCE DE LECHE
SUJI / SEMONILA KI METHAI
Rasmalai
Shahi Kulfa
STRAWBERRY SHAKE
QUICK PERAH/ PEDA- METHAI
Easy Milk Burfi
GAJAR/ CARROT HALWA IN MICROWAVE
KALAKAND SWEET / MITHAI
DOUBLE CHOCOLATE HOT MILK
QUICK COCONUT ROLLS
MILKY GULAB JAMUN
Mocha Chino
STRAWBERRY ICE CREAM SHAKE
Quick Strawberry Ice Cream
CHOCOLATE FUDGE CAKE – FROZEN THEME
BAKERY BISCUITS – CHOCOLATE
Kashmiri Gajraila
DOUBLE CHOCOLATE BROWNIE
Khopra Halwa
MICRO CHOCO FUDGE CAKE
BURNS ROAD KHEER
Mango Kulfi
FILLED CRESCENTS
BAKE WITH S&S – GARLIC BREAD
CARAMEL CREAMY MILK
Homemade Doodh Peda Recipe – Milk Peda Recipe in Urdu
HOME MADE KHOYA
CHINIOTI KHEER
BARFI- MILK FUDGE
NANDOS PERI BITES
EGG LESS GULAB JAMUNS
Walnut Fig Ice Cream
BREAD RASMALAI
CREAMY FRUIT CHAAT
Vanilla Biscuits
Gajar ka Halwa
Gajrela
DOODH PATI CHAI
SHEERMAL
Brownie Fudge Ice Cream
Egg Biscuits
Kesar Peda – Saffron Almond Cookies
CHOCOLATE MOCHA MUG CAKE
DDL GAJAR KI KHER – SOFIA BAIG
ZEBRA CAKE
Sweet Potato Halwa
UMM ALI- EGYPTIA

In [5]:
with open('recipes.json', encoding="utf8") as f:
    data = json.load(f)

df = json_normalize(data[2]['data'])

In [6]:
df.head()

,id,name,ingredients,method,image,video
0,1,KALAKAND SWEET / MITHAI,Milk powder 2 cups~Sugar 1/2 cup or to taste~...,"In a heat resistant bowl add powder milk, crea...",http://www.ainycooks.com/wp-content/uploads/20...,https://www.youtube.com/embed/8ApA0f7vs00
1,2,ARIZONA CHICKEN STEAK,Chicken fillets 2 flattened~Salt to tatse~Whit...,"Marinate chicken fillet with salt, pepper, mus...",http://www.ainycooks.com/wp-content/uploads/20...,None
2,3,CHICKEN CREAMY PIE,Chicken boiled and shredded 1 cup~Onion chopp...,"In a pan add butter and onion, stir it for few...",http://www.ainycooks.com/wp-content/uploads/20...,None
3,4,2 INGREDIENTS CHOCOLATE MOUSSE,Cooking Chocolate / Milk chocolate 200g~Whipp...,Add 1/2 cup cream in chocolate and microwave i...,http://www.ainycooks.com/wp-content/uploads/20...,None
4,5,KARACHI FAMOUS STUDENT BIRYANI,Rice Basmati 1/2 kg~Chicken 1/2 kg~Red chili ...,Soak rice for 20 minutes.~Boil rice and add sa...,http://www.ainycooks.com/wp-content/uploads/20...,None


In [7]:
df['ingredients']

0       Milk powder 2 cups~Sugar  1/2 cup or to taste~...
1       Chicken fillets 2 flattened~Salt to tatse~Whit...
2       Chicken boiled and shredded  1 cup~Onion chopp...
3       Cooking Chocolate / Milk chocolate  200g~Whipp...
4       Rice Basmati 1/2 kg~Chicken  1/2 kg~Red chili ...
                              ...                        
1716    1-1/2 cups  Milk~1/2 teaspoon  Cardamom Powder...
1717    1 cup  Sooji (Semolina/ Rava)~3/4 cup  Sugar~1...
1718    1/2 cup  All Purpose Flour (Maida)~1 cup  Almo...
1719    1/2 cup  Beetroot Puree , steamed and pureed~1...
1720    1 cup  All Purpose Flour (Maida)~1/4 cup  Whol...
Name: ingredients, Length: 1721, dtype: object

In [10]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

datas = []
for index, row in df.iterrows():
    ings = row['ingredients']
    s=re.sub(r'\(.*oz.\)|kg|½|¾|¼|-|crushed|crumbles|ground|minced|tsp|tbsp|required|powder|chopped|sliced|pinch|cups|cup|/|ml|[0-9]',
                             '', 
                             ings)
    
    #Remove Digits
    s=re.sub(r"(\d)", "", s)
    
    #Remove content inside paranthesis
    s=re.sub(r'\([^)]*\)', '', s)
    
    
    s=s.lower()
    
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    s= ' '.join(filtered_sentence)
    
    ings = s.split('~')
    data = {}
    data['recipie'] = row['name']
    data['ingredients'] = ings
    datas.append(data)

In [11]:
df = json_normalize(datas)

In [12]:
df.head()

,recipie,ingredients
0,KALAKAND SWEET / MITHAI,"[milk , sugar taste, cream , nuts]"
1,ARIZONA CHICKEN STEAK,"[chicken fillets flattened, salt tatse, white ..."
2,CHICKEN CREAMY PIE,"[chicken boiled shredded , onion medium , butt..."
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[cooking chocolate milk chocolate g, whipping ..."
4,KARACHI FAMOUS STUDENT BIRYANI,"[rice basmati , chicken , red chili , black pe..."


In [13]:
ings = []
mod = []
for index, row in df.iterrows():
    ings.extend(row['ingredients'])
    mod.append(' '.join(row['ingredients']))

In [14]:
df['ing_mod'] = mod

In [15]:
df.head()

,recipie,ingredients,ing_mod
0,KALAKAND SWEET / MITHAI,"[milk , sugar taste, cream , nuts]",milk sugar taste cream nuts
1,ARIZONA CHICKEN STEAK,"[chicken fillets flattened, salt tatse, white ...",chicken fillets flattened salt tatse white pep...
2,CHICKEN CREAMY PIE,"[chicken boiled shredded , onion medium , butt...",chicken boiled shredded onion medium butter ...
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[cooking chocolate milk chocolate g, whipping ...",cooking chocolate milk chocolate g whipping cream
4,KARACHI FAMOUS STUDENT BIRYANI,"[rice basmati , chicken , red chili , black pe...",rice basmati chicken red chili black pepper...


In [16]:
len(ings)

20177

In [17]:
len(set(ings))

5565

In [149]:
ings = list(set(ings))

In [150]:
with open('ingredients.txt','w') as f:
    for ing in ings:
        f.write(ing.strip()+'\n')

In [163]:
df['ing_mod'] = mod

In [18]:
import pickle

In [19]:
with open('tf_idf','rb') as f:
    new_tf_idf = pickle.load(f)

In [20]:
with open('tf_idf_vec','rb') as f:
    new_tf_idf_vec = pickle.load(f)

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction import text
# from sklearn.feature_extraction.text import TfidfTransformer
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer


query = new_tf_idf_vec.transform(['butter'])
cs = cosine_similarity(query, new_tf_idf)
similarity_list = cs[0]
result_list = []
x = similarity_list
index = np.argwhere(x!=0)
data = {}
for ind in index:
    data[ind[0]] = x[ind][0]

In [49]:
query = new_tf_idf_vec.transform(['chicken'])
cs = cosine_similarity(query, new_tf_idf)
similarity_list = cs[0]
result_list = []
x = similarity_list
index = np.argwhere(x!=0)
data_2 = {}
for ind in index:
    data_2[ind[0]] = x[ind][0]

In [61]:
p_ings= ['milk','chicken']
datas = []
for p in p_ings:
    query = new_tf_idf_vec.transform([p])
    cs = cosine_similarity(query, new_tf_idf)
    similarity_list = cs[0]
    result_list = []
    x = similarity_list
    index = np.argwhere(x!=0)
    data = {}
    for ind in index:
        data[ind[0]] = x[ind][0]
    y = {k: v for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True)}
    datas.append(y)

In [62]:
res = []
for data in datas:
    for k in data.keys():
        res.append(df.iloc[k]['recipie'])

In [63]:
len(res)

878

In [68]:
new_res = list(set([i for i in res if res.count(i)>1]))

In [69]:
len(new_res)

91

In [52]:
res = []
for k in data.keys():
    res.append(df.iloc[k]['recipie'])

In [53]:
res_2 = []
for k in data_2.keys():
    res_2.append(df.iloc[k]['recipie'])

In [55]:
new_res = list(set(res).intersection(res_2))

In [56]:
len(new_res)

61

In [70]:
new_res

['Tandoori Chicken Pizza Recipe in Urdu',
 'Cream of Fresh Asparagus Soup II Recipe in Urdu',
 'Scrumptious Crispy Chicken Broast Recipe in Urdu',
 'Oreo Ice Cream',
 'Pop Corn Chicken (Ready to Cook)',
 'Mango Kulfi',
 'Chinese Style Bun',
 'Chicken Fillet Burger Recipe in Urdu',
 'SPAGHETTI TIKKA PIZZA',
 'Chicken Maryland Recipe in Urdu',
 'Crispy Fried Chicken Burger Recipe in Urdu',
 'Tender Chicken Nuggets',
 'Chicken Yellow Bhuna Recipe in  Urdu',
 'Homemade Oreo Ice cream without Machine',
 'CREAMY CHICKEN POTATO ROLLS',
 'NANDOS PERI BITES',
 'Thumb Print Cookies',
 'Prawn Manchurian with Noodles Recipe in Urdu',
 'Chicken Harissa',
 'Tandori Chicken Masala',
 'Pasta Payasam Recipe – Fusion Payasam With Pasta',
 'QUICK MALAI TIKKA PIZZA',
 'Special Crunchy Fillet Burger Recipe in Urdu',
 'Italian Chicken Puffs',
 'Zinger Style Burger Recipe in Urdu',
 'Spanish Chicken Pulao Recipe in Urdu',
 'Oatmeal in A Mug',
 'Buffalo Chicken Tenders Recipe in Urdu',
 'Beef Baking Roll Reci